In [1]:
using Random
using SCS
using Test
using BenchmarkTools
using MathOptInterface
using SparseArrays
using MatrixOptInterface
using DiffOpt

const MOI  = MathOptInterface
const MatOI = MatrixOptInterface
const MOIU = MathOptInterface.Utilities;

const ATOL=1e-3
const RTOL=1e-3;

┌ Info: Recompiling stale cache file /home/pika/.julia/compiled/v1.0/SCS/uaibp.ji for SCS [c946c3f1-0d1f-5ce8-9dea-7daa1f7e2d13]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /home/pika/.julia/compiled/v1.0/MatrixOptInterface/TIqsU.ji for MatrixOptInterface [2f4eb8e6-3e35-4ae4-8c7a-f3d7d9bf20ed]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /home/pika/.julia/compiled/v1.0/DiffOpt/xRKO7.ji for DiffOpt [930fe3bc-9c6b-11ea-2d94-6184641e85e7]
└ @ Base loading.jl:1190


In [2]:
# starting w/ a simple LP

D = 10  # variable dimension
M = 50  # no of equality constraints
N = 200; # no of inequality constraints

In [3]:
x̂ = rand(D) # solution

c = rand(D) # objective coeffs

A = rand(M, D) # equality part
b = A*x̂

G = rand(N, D) # inequality part
h = G*x̂ .+ rand(N);

In [4]:
model = diff_optimizer(SCS.Optimizer)
x = MOI.add_variables(model, D)

# define objective
objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(), objective_function)
MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)

# set equality constraints
# # push!(constraint_indices, MOI.add_constraint(model,MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(A[i,:], x), 0.),MOI.LessThan(b[i])))
ceq = [ 
    MOI.add_constraint(
        model,
        MOI.VectorAffineFunction(
            MOI.VectorAffineTerm.(1:1, MOI.ScalarAffineTerm.(A[i,:], x)),
            [-b[i]]
        ),
        MOI.Zeros(1)
    ) for i in 1:M   
]

# set inequality constraints
cineq = [ 
    MOI.add_constraint(
        model,
        MOI.VectorAffineFunction(
            MOI.VectorAffineTerm.(1:1, MOI.ScalarAffineTerm.(-G[i,:], x)),
            [h[i]]
        ),
        MOI.Nonnegatives(1)
    ) for i in 1:N   
]

MOI.set(model, MOI.Silent(), true)
MOI.optimize!(model)

x̄ = MOI.get(model, MOI.VariablePrimal(), x);  # solution

In [5]:
# testing solution validity
@test (c'x̄ <= c'x̂) || c' * (x̄ - x̂) <= ATOL  
@test A*x̄ ≈ b atol=ATOL rtol=RTOL
@test G*x̄ <= h

Test Passed

In [6]:
dA = A #ones(M, D)
db = b #ones(M)

for i in 1:M
    for j in 1:D
        MOI.set(
            model,
            DiffOpt.ForwardIn{DiffOpt.ConstraintCoefficient}(),
            x[j],
            ceq[i],
            [dA[i, j]]
        )
    end
    MOI.set(
        model,
        DiffOpt.ForwardIn{DiffOpt.ConstraintConstant}(),
        ceq[i],
        [db[i]]
    )
end

@benchmark DiffOpt.forward(model)

BenchmarkTools.Trial: 
  memory estimate:  468.66 KiB
  allocs estimate:  1694
  --------------
  minimum time:     868.454 μs (0.00% GC)
  median time:      1.021 ms (0.00% GC)
  mean time:        1.392 ms (15.63% GC)
  maximum time:     6.461 ms (65.58% GC)
  --------------
  samples:          3574
  evals/sample:     1

In [ ]:
dx = MOI.get.(
    model,
    DiffOpt.ForwardOut{MOI.VariablePrimal}(),
    x
)